In [1]:
import fredapi as fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import requests
import os

# Set the API key

FRED_API_KEY = os.getenv('FRED_API_KEY')

# Create a FRED API object
fred_api = fred.Fred(api_key=FRED_API_KEY)

# Get the release dates for all releases
release_dates = fred_api.get_series_all_releases('UNRATE')

In [2]:
import sys

# 현재 작업 디렉토리 설정
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

# 데이터 파일 경로 설정
data_path = os.path.join('../data', 'nowcast-model-variables.xlsx')

variables = pd.read_excel(data_path, engine='openpyxl', sheet_name='variables')
releases = pd.read_excel(data_path, engine='openpyxl', sheet_name='releases')

variables.to_csv('../data/nowcast-variables.csv')
releases.to_csv('../data/nowcast-releases.csv')

---

In [153]:
import requests
import pandas as pd

API_KEY = FRED_API_KEY
BASE_URL = 'https://api.stlouisfed.org/fred'

source_keys = pd.read_csv('../data/nowcast-variables.csv')

# hist_source가 fred인 경우
fred_keys = source_keys[source_keys['hist_source'] == 'fred']['hist_source_key']

# hist_source가 yahoo인 경우
yf_tickers = source_keys[source_keys['hist_source'] == 'yahoo']['hist_source_key']

# yf_tickers의 종가를 yfinacne로 다운로드하여 병합
import yfinance as yf

data_dict = {}
for ticker in yf_tickers:
    data = yf.download(ticker, start='2000-01-01', end='2024-12-31')
    data = data['Close']
    data.name = ticker
    data_dict[ticker] = data

market_df = pd.concat(data_dict.values(), axis=1)
market_df.columns = data_dict.keys()
market_df.index.name = 'Date'

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [154]:
meta_dict = {}
data_dict = {}
skipped_keys = []

for source_key in fred_keys:
    print(f"Processing {source_key}...")
    try:
        # 메타데이터 수집
        meta_url = f'{BASE_URL}/series'
        meta_params = {'api_key': API_KEY, 'file_type': 'json', 'series_id': source_key}
        meta_r = requests.get(meta_url, params=meta_params).json().get('seriess', [])
        if not meta_r:
            print(f"skip {source_key}: No metadata available")
            skipped_keys.append((source_key, "No metadata"))
            continue
        meta_df = pd.DataFrame(meta_r)
        meta_dict[source_key] = meta_df

        # release id
        release_url = f'{BASE_URL}/series/release'
        release_params = {'api_key': API_KEY, 'file_type': 'json', 'series_id': source_key}
        release_response = requests.get(release_url, params=release_params).json()
        if 'releases' not in release_response or not release_response['releases']:
            print(f"skip {source_key}: No release info")
            skipped_keys.append((source_key, "No release info"))
            continue
        release_id = release_response['releases'][0]['id']

        # obs
        obs_url = f'{BASE_URL}/series/observations'
        obs_params = {'api_key': API_KEY, 'file_type': 'json', 'series_id': source_key}
        obs_response = requests.get(obs_url, params=obs_params).json()
        if 'observations' not in obs_response:
            print(f"skip {source_key}: No observations")
            skipped_keys.append((source_key, "No observations"))
            continue
        obs_data = obs_response['observations']
        if not obs_data:
            print(f"skip {source_key}: Empty observations")
            skipped_keys.append((source_key, "Empty observations"))
            continue

        # release dates
        rel_url = f'{BASE_URL}/release/dates'
        rel_params = {'api_key': API_KEY, 'file_type': 'json', 'release_id': release_id}
        rel_data = requests.get(rel_url, params=rel_params).json()['release_dates']
        if not rel_data:
            print(f"skip {source_key}: No release dates")
            skipped_keys.append((source_key, "No release dates"))
            continue

        # obs_df
        obs_df = pd.DataFrame(obs_data)
        obs_df['date'] = pd.to_datetime(obs_df['date'], format='%Y-%m-%d', errors='coerce')
        obs_df['base'] = (obs_df['date'] + pd.DateOffset(months=1)).dt.to_period('M').astype(str)
        obs_df[source_key] = obs_df['value']
        obs_df = obs_df[['base', source_key]]

        # release_df
        rel_df = pd.DataFrame(rel_data)
        rel_df['release_date'] = pd.to_datetime(rel_df['date'], format='%Y-%m-%d', errors='coerce')
        rel_df['base'] = rel_df['release_date'].dt.to_period('M').astype(str)
        rel_df = rel_df[['release_date', 'base']]

        # merge + 중복 제거 + 리샘플링
        temp_df = obs_df.merge(rel_df, on='base', how='left')  # inner 대신 left로 변경
        temp_df = temp_df.drop(columns=['base']).set_index('release_date')
        temp_df = temp_df.groupby('release_date').first()  # 중복 제거
        temp_df = temp_df.resample('D').asfreq()

        data_dict[source_key] = temp_df

    except Exception as e:
        print(f"skip {source_key}: {e}")
        skipped_keys.append((source_key, str(e)))
        continue

# 데이터 병합
merged_df = pd.concat(data_dict.values(), axis=1, join='outer')
meta_full_df = pd.concat(meta_dict.values(), axis=0, ignore_index=True)

# 결과 출력
print("메타데이터 df shape:", meta_full_df.shape)
print("merged_df shape:", merged_df.shape)
print(meta_full_df.head())
print(merged_df.head())
print("\nSkipped keys:")
for key, reason in skipped_keys:
    print(f"{key}: {reason}")

Processing PCEC96...
Processing A068RC1...
Processing PMSAVE...
Processing PSAVERT...
Processing RPI...
Processing DSPIC96...
Processing TOTCI...
Processing RELACBW027SBOG...
Processing DRSFRMACBS...
Processing DRCCLACBS...
Processing DRBLACBS...
Processing GDPC1...
Processing PCECC96...
Processing DGDSRX1Q020SBEA...
Processing PCDGCC96...
Processing DMOTRX1Q020SBEA...
Processing DFDHRX1Q020SBEA...
Processing DREQRX1Q020SBEA...
Processing DODGRX1Q020SBEA...
Processing PCNDGC96...
Processing DFXARX1Q020SBEA...
Processing DCLORX1Q020SBEA...
Processing DGOERX1Q020SBEA...
Processing DONGRX1Q020SBEA...
Processing PCESVC96...
Processing DHUTRX1Q020SBEA...
Processing DHLCRX1Q020SBEA...
Processing DTRSRX1Q020SBEA...
Processing DRCARX1Q020SBEA...
Processing DFSARX1Q020SBEA...
Processing DIFSRX1Q020SBEA...
Processing DOTSRX1Q020SBEA...
Processing DNPIRX1Q020SBEA...
Processing GPDIC1...
Processing PNFIC1...
Processing B009RX1Q020SBEA...
Processing Y033RX1Q020SBEA...
Processing Y001RX1Q020SBEA...


In [165]:
source_key = "DCOILWTICO"
meta_url = f'{BASE_URL}/series'
meta_params = {'api_key': API_KEY, 'file_type': 'json', 'series_id': source_key}
response = requests.get(meta_url, params=meta_params)
print(f"Status: {response.status_code}")
print(response.json())

Status: 200
{'realtime_start': '2025-03-19', 'realtime_end': '2025-03-19', 'seriess': [{'id': 'DCOILWTICO', 'realtime_start': '2025-03-19', 'realtime_end': '2025-03-19', 'title': 'Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma', 'observation_start': '1986-01-02', 'observation_end': '2025-03-10', 'frequency': 'Daily', 'frequency_short': 'D', 'units': 'Dollars per Barrel', 'units_short': '$ per Barrel', 'seasonal_adjustment': 'Not Seasonally Adjusted', 'seasonal_adjustment_short': 'NSA', 'last_updated': '2025-03-12 12:10:03-05', 'popularity': 77, 'notes': 'Definitions, Sources and Explanatory Notes (http://www.eia.doe.gov/dnav/pet/TblDefs/pet_pri_spt_tbldef2.asp)'}]}


In [ ]:
import pandas as pd
import requests
import time
import yfinance as yf

API_KEY = FRED_API_KEY
BASE_URL = 'https://api.stlouisfed.org/fred'

source_keys = pd.read_csv('../data/nowcast-variables.csv')
fred_keys = source_keys[source_keys['hist_source'] == 'fred']['hist_source_key']
yf_tickers = source_keys[source_keys['hist_source'] == 'yahoo']['hist_source_key']

def fetch_yahoo_data(tickers, start='2000-01-01', end='2024-12-31'):
    data_dict = {}
    for ticker in tickers:
        print(f"Fetching Yahoo data for {ticker}...")
        data = yf.download(ticker, start=start, end=end)
        data = data['Close']
        data.name = ticker
        data_dict[ticker] = data
    market_df = pd.concat(data_dict.values(), axis=1)
    market_df.columns = data_dict.keys()
    market_df.index.name = 'Date'
    return market_df

def fetch_fred_series(source_key, api_key, base_url, timeout=10):
    try:
        # 메타데이터
        meta_url = f'{base_url}/series'
        meta_params = {'api_key': api_key, 'file_type': 'json', 'series_id': source_key}
        meta_response = requests.get(meta_url, params=meta_params, timeout=timeout)
        meta_json = meta_response.json()
        meta_r = meta_json.get('seriess', [])
        if not meta_r:
            return None, None, f"No metadata available - Response: {meta_json}"

        meta_df = pd.DataFrame(meta_r)
        frequency = meta_df['frequency_short'].iloc[0]  # 주기: D, W, M, Q 등

        release_url = f'{base_url}/series/release'
        release_params = {'api_key': api_key, 'file_type': 'json', 'series_id': source_key}
        release_response = requests.get(release_url, params=release_params, timeout=timeout)
        release_json = release_response.json()
        if 'releases' not in release_json or not release_json['releases']:
            return None, None, "No release info"
        release_id = release_json['releases'][0]['id']

        obs_url = f'{base_url}/series/observations'
        obs_params = {'api_key': api_key, 'file_type': 'json', 'series_id': source_key}
        obs_response = requests.get(obs_url, params=obs_params, timeout=timeout)
        obs_json = obs_response.json()
        if 'observations' not in obs_json or not obs_json['observations']:
            return None, None, "No observations or empty"

        obs_df = pd.DataFrame(obs_json['observations'])
        obs_df['date'] = pd.to_datetime(obs_df['date'], format='%Y-%m-%d', errors='coerce')
        obs_df[source_key] = obs_df['value']

        # 주기에 따라 날짜 조정
        if frequency == 'M':  # 월간
            obs_df['adjusted_date'] = obs_df['date'] + pd.offsets.MonthEnd(0) + pd.offsets.MonthBegin(1)
        elif frequency == 'Q':  # 분기
            obs_df['adjusted_date'] = obs_df['date'] + pd.offsets.QuarterEnd(0) + pd.offsets.MonthBegin(2)  # 2개월 뒤
        elif frequency == 'W':  # 주간
            obs_df['adjusted_date'] = obs_df['date'] + pd.offsets.Week(1)  # 1주 뒤
        else:
            obs_df['adjusted_date'] = obs_df['date']  # 기타 주기

        obs_df = obs_df[['adjusted_date', source_key]].set_index('adjusted_date')

        rel_url = f'{base_url}/release/dates'
        rel_params = {'api_key': api_key, 'file_type': 'json', 'release_id': release_id}
        rel_response = requests.get(rel_url, params=rel_params, timeout=timeout)
        rel_data = rel_response.json()['release_dates']
        if not rel_data:
            return None, None, "No release dates"

        rel_df = pd.DataFrame(rel_data)
        rel_df['release_date'] = pd.to_datetime(rel_df['date'], format='%Y-%m-%d', errors='coerce')
        rel_df = rel_df[['release_date']].sort_values('release_date')

        temp_df = obs_df.reset_index()
        temp_df = pd.merge_asof(temp_df, rel_df, left_on='adjusted_date', right_on='release_date', direction='forward')
        temp_df = temp_df.drop(columns=['adjusted_date'])
        temp_df = temp_df.groupby('release_date').first()
        temp_df = temp_df.resample('D').asfreq()

        return meta_df, temp_df, None

    except Exception as e:
        return None, None, str(e)

def fetch_fred_data(keys, api_key, base_url, delay_seconds=0.3):
    meta_dict = {}
    data_dict = {}
    skipped_keys = []

    for source_key in keys:
        print(f"Processing {source_key}...")
        meta_df, series_df, error = fetch_fred_series(source_key, api_key, base_url)
        if error:
            print(f"skip {source_key}: {error}")
            skipped_keys.append((source_key, error))
        else:
            meta_dict[source_key] = meta_df
            data_dict[source_key] = series_df
        time.sleep(delay_seconds)

    return meta_dict, data_dict, skipped_keys

# 실행
market_df = fetch_yahoo_data(yf_tickers)

mid_point = len(fred_keys) // 2
first_half = fred_keys[:mid_point]
second_half = fred_keys[mid_point:]

print("Fetching first half of FRED keys...")
meta_dict1, data_dict1, skipped1 = fetch_fred_data(first_half, API_KEY, BASE_URL)

print("Waiting before second half...")
time.sleep(60)

print("Fetching second half of FRED keys...")
meta_dict2, data_dict2, skipped2 = fetch_fred_data(second_half, API_KEY, BASE_URL)

meta_dict = {**meta_dict1, **meta_dict2}
data_dict = {**data_dict1, **data_dict2}
skipped_keys = skipped1 + skipped2

fred_merged_df = pd.concat(data_dict.values(), axis=1, join='outer')
fred_meta_full_df = pd.concat(meta_dict.values(), axis=0, ignore_index=True)
final_df = pd.concat([market_df, fred_merged_df], axis=1)

print("FRED 메타데이터 df shape:", fred_meta_full_df.shape)
print("FRED merged_df shape:", fred_merged_df.shape)
print("Market df shape:", market_df.shape)
print("Final df shape:", final_df.shape)
print(fred_meta_full_df.head())
print(final_df.head())
print("\nSkipped keys:")
for key, reason in skipped_keys:
    print(f"{key}: {reason}")

# 주기별 데이터 확인
print("\nCPIAUCSL (월간):")
print(final_df[final_df['CPIAUCSL'].notna()][['CPIAUCSL']])
print("\nGASREGW (주간):")
print(final_df[final_df['GASREGW'].notna()][['GASREGW']])
print("\nGDPC1 (분기, 예시):")
print(final_df[final_df['GDPC1'].notna()][['GDPC1']] if 'GDPC1' in final_df.columns else "GDPC1 not in data")

In [ ]:
# 필수 FRED series만 필터링
fred_keys = [
    'CPIAUCSL',  # CPI headline
    'CPILFESL',  # Core CPI
    'CPIUFDSL',  # CPI Food
    'CPIHOSSL',  # CPI Food at home
    'CUSR0000SETB01',  # CPI Gasoline
    'PCEPI',  # PCE price index
    'PCEPILFE',  # Core PCE price index
    'DPCERD3A086NBEA',  # PCE food off-premises
    'GASREGW',  # Weekly gasoline price
    'DCOILWTICO'  # Crude oil (daily WTI 기준)
]


# 이후 fetch_fred_data() 부분 동일하게 실행

import pandas as pd
import requests
import time

API_KEY = FRED_API_KEY
BASE_URL = 'https://api.stlouisfed.org/fred'

def fetch_fred_series(source_key, api_key, base_url, timeout=10):
    try:
        # 메타데이터
        meta_url = f'{base_url}/series'
        meta_params = {'api_key': api_key, 'file_type': 'json', 'series_id': source_key}
        meta_response = requests.get(meta_url, params=meta_params, timeout=timeout)
        meta_json = meta_response.json()
        meta_r = meta_json.get('seriess', [])
        if not meta_r:
            return None, None, f"No metadata available - Response: {meta_json}"

        meta_df = pd.DataFrame(meta_r)
        frequency = meta_df['frequency_short'].iloc[0]  # 주기: D, W, M, Q 등

        release_url = f'{base_url}/series/release'
        release_params = {'api_key': api_key, 'file_type': 'json', 'series_id': source_key}
        release_response = requests.get(release_url, params=release_params, timeout=timeout)
        release_json = release_response.json()
        if 'releases' not in release_json or not release_json['releases']:
            return None, None, "No release info"
        release_id = release_json['releases'][0]['id']

        obs_url = f'{base_url}/series/observations'
        obs_params = {'api_key': api_key, 'file_type': 'json', 'series_id': source_key}
        obs_response = requests.get(obs_url, params=obs_params, timeout=timeout)
        obs_json = obs_response.json()
        if 'observations' not in obs_json or not obs_json['observations']:
            return None, None, "No observations or empty"

        obs_df = pd.DataFrame(obs_json['observations'])
        obs_df['date'] = pd.to_datetime(obs_df['date'], format='%Y-%m-%d', errors='coerce')
        obs_df[source_key] = obs_df['value']

        # 주기에 따라 날짜 조정
        if frequency == 'M':  # 월간
            obs_df['adjusted_date'] = obs_df['date'] + pd.offsets.MonthEnd(0) + pd.offsets.MonthBegin(1)
        elif frequency == 'Q':  # 분기
            obs_df['adjusted_date'] = obs_df['date'] + pd.offsets.QuarterEnd(0) + pd.offsets.MonthBegin(2)  # 2개월 뒤
        elif frequency == 'W':  # 주간
            obs_df['adjusted_date'] = obs_df['date'] + pd.offsets.Week(1)  # 1주 뒤
        else:
            obs_df['adjusted_date'] = obs_df['date']  # 기타 주기

        obs_df = obs_df[['adjusted_date', source_key]].set_index('adjusted_date')

        rel_url = f'{base_url}/release/dates'
        rel_params = {'api_key': api_key, 'file_type': 'json', 'release_id': release_id}
        rel_response = requests.get(rel_url, params=rel_params, timeout=timeout)
        rel_data = rel_response.json()['release_dates']
        if not rel_data:
            return None, None, "No release dates"

        rel_df = pd.DataFrame(rel_data)
        rel_df['release_date'] = pd.to_datetime(rel_df['date'], format='%Y-%m-%d', errors='coerce')
        rel_df = rel_df[['release_date']].sort_values('release_date')

        temp_df = obs_df.reset_index()
        temp_df = pd.merge_asof(temp_df, rel_df, left_on='adjusted_date', right_on='release_date', direction='forward')
        temp_df = temp_df.drop(columns=['adjusted_date'])
        temp_df = temp_df.groupby('release_date').first()
        temp_df = temp_df.resample('D').asfreq()

        return meta_df, temp_df, None

    except Exception as e:
        return None, None, str(e)

def fetch_fred_data(keys, api_key, base_url, delay_seconds=0.1):
    meta_dict = {}
    data_dict = {}
    skipped_keys = []

    for source_key in keys:
        print(f"Processing {source_key}...")
        meta_df, series_df, error = fetch_fred_series(source_key, api_key, base_url)
        if error:
            print(f"skip {source_key}: {error}")
            skipped_keys.append((source_key, error))
        else:
            meta_dict[source_key] = meta_df
            data_dict[source_key] = series_df
        time.sleep(delay_seconds)

    return meta_dict, data_dict, skipped_keys

# 실행

mid_point = len(fred_keys) // 2
first_half = fred_keys[:mid_point]
second_half = fred_keys[mid_point:]

print("Fetching first half of FRED keys...")
meta_dict1, data_dict1, skipped1 = fetch_fred_data(first_half, API_KEY, BASE_URL)

print("Waiting before second half...")

print("Fetching second half of FRED keys...")
meta_dict2, data_dict2, skipped2 = fetch_fred_data(second_half, API_KEY, BASE_URL)

meta_dict = {**meta_dict1, **meta_dict2}
data_dict = {**data_dict1, **data_dict2}
skipped_keys = skipped1 + skipped2

fred_merged_df = pd.concat(data_dict.values(), axis=1, join='outer')
fred_meta_full_df = pd.concat(meta_dict.values(), axis=0, ignore_index=True)

print("FRED 메타데이터 df shape:", fred_meta_full_df.shape)
print("FRED merged_df shape:", fred_merged_df.shape)
print(fred_meta_full_df.head())
print("\nSkipped keys:")
for key, reason in skipped_keys:
    print(f"{key}: {reason}")

Fetching first half of FRED keys...
Processing CPIAUCSL...
Processing CPILFESL...
Processing CPIUFDSL...
Processing CPIHOSSL...
Processing CUSR0000SETB01...
Waiting before second half...
Fetching second half of FRED keys...
Processing PCEPI...


In [188]:
final_df.to_csv('../data/final_df.csv')
fred_meta_full_df.to_csv('../data/fred_meta_full_df.csv')

## 주요 거시경제 지표의 메타 정보를 인기 순으로 다운로드

In [85]:
# Step 1: 모든 소스 가져오기
source_url = f'{BASE_URL}/sources'
source_params = {'api_key': API_KEY, 'file_type': 'json'}
sources = requests.get(source_url, params=source_params).json()['sources']
source_ids = [s['id'] for s in sources]

all_meta = []

# Step 2: 소스별 시리즈 수집
for sid in source_ids:
    series_url = f'{BASE_URL}/source/releases'
    series_params = {
        'api_key': API_KEY,
        'file_type': 'json',
        'source_id': sid
    }
    releases = requests.get(series_url, params=series_params).json().get('releases', [])
    for release in releases:
        rel_id = release['id']
        series_fetch_url = f'{BASE_URL}/release/series'
        series_fetch_params = {
            'api_key': API_KEY,
            'file_type': 'json',
            'release_id': rel_id,
            'order_by': 'popularity',
            'sort_order': 'desc',
            'limit': 1000
        }
        series_data = requests.get(series_fetch_url, params=series_fetch_params).json()
        all_meta.extend(series_data.get('seriess', []))

# Step 3: DataFrame 변환 및 필터링
meta_df = pd.DataFrame(all_meta)
meta_df = meta_df.drop_duplicates(subset=['id'])
meta_df = meta_df[meta_df['popularity'] >= 10].sort_values('popularity', ascending=False).reset_index(drop=True)

print(meta_df[['id', 'title', 'popularity']])

                     id                                              title  \
0                T10Y2Y  10-Year Treasury Constant Maturity Minus 2-Yea...   
1              FEDFUNDS                       Federal Funds Effective Rate   
2              CPIAUCSL  Consumer Price Index for All Urban Consumers: ...   
3                UNRATE                                  Unemployment Rate   
4                T10Y3M  10-Year Treasury Constant Maturity Minus 3-Mon...   
...                 ...                                                ...   
5275  BOGZ1FL263061145Q  Rest of the World; Treasury Securities Held by...   
5276  BOGZ1LM503062003Q  Other Financial Business; Municipal Securities...   
5277           FFWSJLOW  Low Value of the Federal Funds Rate for the In...   
5278            FFHTLOW  Low Value of the Federal Funds Rate for the In...   
5279             WUIBRA                 World Uncertainty Index for Brazil   

      popularity  
0            100  
1             98  
2     